# automate one column

In [19]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# Parameters
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "1Esy8YW-t2B3-8uwhaFrDD2-1fgzcXJ31hyi97DVPKXI"  # raw sheet to update from
CREDENTIALS_FILE = "client_secret_70065301055-fjcq0nugl0id8k0oab6qjse8e3o24in6.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"
RAW_SHEET_NAME = "raw"
TARGET_SHEET_NAME = "BatchAudioSummary"
FORMULA_COLUMN = "E"
COL_TO_UPDATE = "C"

STATUS_DATA = [
    'Raw Delivered',
    'Delivered greater than acceptance threshold',
    'Raw Redelivery',
    'Redelivered greater than acceptance threshold',
    'Accepted post Initial Check (file level)',
    'Accepted post Initial check (chunk level)',
    'Accepted post automated single audio check (chunk level)',
    'Delivered for manual QC',
    'Accepted post final single Audio Manual QC (chunk level)'
]

# Function to authenticate and create a Google Sheets API service instance
def get_sheets_service():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        return service
    except HttpError as err:
        print(err)
        return None

# Function to check if a sheet exists
def sheet_exists(service, spreadsheet_id, sheet_name):
    try:
        sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        for sheet in sheets:
            if sheet.get("properties", {}).get("title") == sheet_name:
                return True
        return False
    except HttpError as err:
        print(err)
        return False

# Function to create a new sheet and add data
def create_or_update_sheet(service, spreadsheet_id, sheet_name):
    # Create a DataFrame with the status data
    df = pd.DataFrame({'Status': STATUS_DATA})

    # Convert DataFrame to list of lists
    values = [df.columns.tolist()] + df.values.tolist()

    # Check if the sheet exists
    if not sheet_exists(service, spreadsheet_id, sheet_name):
        # Create a new sheet named 'BatchAudioSummary'
        add_sheet_request = {
            "requests": [
                {
                    "addSheet": {
                        "properties": {
                            "title": sheet_name
                        }
                    }
                }
            ]
        }
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=add_sheet_request).execute()

    # Write the DataFrame to the sheet
    body = {
        "values": values
    }
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!A1",
        valueInputOption="RAW",
        body=body
    ).execute()

    # Read the header of the formula column from the 'raw' sheet
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=f"{RAW_SHEET_NAME}!{FORMULA_COLUMN}2"  # 2 becuase in raw sheet the header is in the 2nd row
    ).execute()
    raw_header = result.get("values", [[]])[0][0]

    # Update the header for the new column in 'BatchAudioSummary'
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!{COL_TO_UPDATE}1", # 1 beacuse we are adding the header of raw in the 1st row
        valueInputOption="RAW",
        body={"values": [[raw_header]]}
    ).execute()

    # Add formulas to the new column
    for i in range(2, 11):
        if i == 9:
            formula = f'={COL_TO_UPDATE}7-{COL_TO_UPDATE}8'
            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{COL_TO_UPDATE}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()

        elif i == 10:
            formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{FORMULA_COLUMN}{i}:{FORMULA_COLUMN}1000)-ROW({RAW_SHEET_NAME}!{FORMULA_COLUMN}{i}),8)=0)*{RAW_SHEET_NAME}!{FORMULA_COLUMN}{i}:{FORMULA_COLUMN}1000)/60'
            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{COL_TO_UPDATE}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()
        else :
            formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{FORMULA_COLUMN}{i+1}:{FORMULA_COLUMN}1000)-ROW({RAW_SHEET_NAME}!{FORMULA_COLUMN}{i+1}),8)=0)*{RAW_SHEET_NAME}!{FORMULA_COLUMN}{i+1}:{FORMULA_COLUMN}1000)/60'
            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{COL_TO_UPDATE}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()
        

def main():
    service = get_sheets_service()
    if service:
        create_or_update_sheet(service, SPREADSHEET_ID, TARGET_SHEET_NAME)

if __name__ == "__main__":
    main()


# automate all the columns


In [ ]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# Parameters
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "1Esy8YW-t2B3-8uwhaFrDD2-1fgzcXJ31hyi97DVPKXI"  # raw sheet to update from
CREDENTIALS_FILE = "client_secret_70065301055-fjcq0nugl0id8k0oab6qjse8e3o24in6.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"
RAW_SHEET_NAME = "raw"
TARGET_SHEET_NAME = "BatchAudioSummaryAuto"
START_FORMULA_COLUMN = ord('E')  # Start column 'E' in ASCII
END_FORMULA_COLUMN = ord('Z')  # End column 'Z' in ASCII
START_UPDATE_COLUMN = ord('C')  # Start column 'C' in ASCII

STATUS_DATA = [
    'Raw Delivered',
    'Delivered greater than acceptance threshold',
    'Raw Redelivery',
    'Redelivered greater than acceptance threshold',
    'Accepted post Initial Check (file level)',
    'Accepted post Initial check (chunk level)',
    'Accepted post automated single audio check (chunk level)',
    'Delivered for manual QC',
    'Accepted post final single Audio Manual QC (chunk level)'
]

# Function to authenticate and create a Google Sheets API service instance
def get_sheets_service():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        return service
    except HttpError as err:
        print(err)
        return None

# Function to check if a sheet exists
def sheet_exists(service, spreadsheet_id, sheet_name):
    try:
        sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        for sheet in sheets:
            if sheet.get("properties", {}).get("title") == sheet_name:
                return True
        return False
    except HttpError as err:
        print(err)
        return False

# Function to create a new sheet and add data
def create_or_update_sheet(service, spreadsheet_id, sheet_name):
    # Create a DataFrame with the status data
    df = pd.DataFrame({'Status': STATUS_DATA})

    # Convert DataFrame to list of lists
    values = [df.columns.tolist()] + df.values.tolist()

    # Check if the sheet exists
    if not sheet_exists(service, spreadsheet_id, sheet_name):
        # Create a new sheet named 'BatchAudioSummary'
        add_sheet_request = {
            "requests": [
                {
                    "addSheet": {
                        "properties": {
                            "title": sheet_name
                        }
                    }
                }
            ]
        }
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=add_sheet_request).execute()

    # Write the DataFrame to the sheet
    body = {
        "values": values
    }
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!A1",
        valueInputOption="RAW",
        body=body
    ).execute()

    # Loop through columns from 'E' to 'Z' for FORMULA_COLUMN and 'C' onwards for COL_TO_UPDATE
    for col_offset in range(END_FORMULA_COLUMN - START_FORMULA_COLUMN + 1):
        formula_column = chr(START_FORMULA_COLUMN + col_offset)
        col_to_update = chr(START_UPDATE_COLUMN + col_offset)

        # Read the header of the formula column from the 'raw' sheet
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=f"{RAW_SHEET_NAME}!{formula_column}2"  # 2 because in raw sheet the header is in the 2nd row
        ).execute()
        raw_header = result.get("values", [[]])[0][0]

        # Update the header for the new column in 'BatchAudioSummary'
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!{col_to_update}1",  # 1 because we are adding the header of raw in the 1st row
            valueInputOption="RAW",
            body={"values": [[raw_header]]}
        ).execute()

        # Add formulas to the new column
        for i in range(2, 11):
            if i == 9:
                formula = f'={col_to_update}7-{col_to_update}8'
            elif i == 10:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)/60'
            else:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i+1}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)/60'

            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{col_to_update}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()

def main():
    service = get_sheets_service()
    if service:
        create_or_update_sheet(service, SPREADSHEET_ID, TARGET_SHEET_NAME)

if __name__ == "__main__":
    main()


# automate all columns with some delay between the updates

In [1]:
import os
import time
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# Parameters
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "1Esy8YW-t2B3-8uwhaFrDD2-1fgzcXJ31hyi97DVPKXI"  # raw sheet to update from
CREDENTIALS_FILE = "client_secret_70065301055-fjcq0nugl0id8k0oab6qjse8e3o24in6.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"
RAW_SHEET_NAME = "rawAuto"
TARGET_SHEET_NAME = "BatchAudioSummaryAuto"
START_FORMULA_COLUMN = ord('E')  # Start column 'E' in ASCII
END_FORMULA_COLUMN = ord('Z')  # End column 'Z' in ASCII
START_UPDATE_COLUMN = ord('C')  # Start column 'C' in ASCII
RATE_LIMIT_DELAY = 0.5  # Delay in seconds between requests

STATUS_DATA = [
    'Raw Delivered',
    'Delivered greater than acceptance threshold',
    'Raw Redelivery',
    'Redelivered greater than acceptance threshold',
    'Accepted post Initial Check (file level)',
    'Accepted post Initial check (chunk level)',
    'Accepted post automated single audio check (chunk level)',
    'Delivered for manual QC',
    'Accepted post final single Audio Manual QC (chunk level)'
]

# Function to authenticate and create a Google Sheets API service instance
def get_sheets_service():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        return service
    except HttpError as err:
        print(err)
        return None

# Function to check if a sheet exists
def sheet_exists(service, spreadsheet_id, sheet_name):
    try:
        sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        for sheet in sheets:
            if sheet.get("properties", {}).get("title") == sheet_name:
                return True
        return False
    except HttpError as err:
        print(err)
        return False

# Function to create a new sheet and add data
def create_or_update_sheet(service, spreadsheet_id, sheet_name):
    # Create a DataFrame with the status data
    df = pd.DataFrame({'Status': STATUS_DATA})

    # Convert DataFrame to list of lists
    values = [df.columns.tolist()] + df.values.tolist()

    # Check if the sheet exists
    if not sheet_exists(service, spreadsheet_id, sheet_name):
        # Create a new sheet named 'BatchAudioSummary'
        add_sheet_request = {
            "requests": [
                {
                    "addSheet": {
                        "properties": {
                            "title": sheet_name
                        }
                    }
                }
            ]
        }
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=add_sheet_request).execute()

    # Write the DataFrame to the sheet
    body = {
        "values": values
    }
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!A1",
        valueInputOption="RAW",
        body=body
    ).execute()
    time.sleep(RATE_LIMIT_DELAY)

    # Determine the number of columns in the 'raw' sheet dynamically
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=f"{RAW_SHEET_NAME}!2:2"  # Read the second row to get column headers
    ).execute()
    raw_headers = result.get("values", [])
    if raw_headers:
        num_columns_raw = len(raw_headers[0])
    else:
        num_columns_raw = 0

    # Loop through columns from 'E' to 'Z' for FORMULA_COLUMN and 'C' onwards for COL_TO_UPDATE
    for col_offset in range(min(END_FORMULA_COLUMN - START_FORMULA_COLUMN + 1, num_columns_raw)):
        formula_column = chr(START_FORMULA_COLUMN + col_offset)
        col_to_update = chr(START_UPDATE_COLUMN + col_offset)

        # Read the header of the formula column from the 'raw' sheet
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=f"{RAW_SHEET_NAME}!{formula_column}2"  # 2 because in raw sheet the header is in the 2nd row
        ).execute()
        raw_header = result.get("values", [[]])[0][0] if result.get("values") and len(result.get("values")) > 0 else ""

        # Update the header for the new column in 'BatchAudioSummary'
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!{col_to_update}1",  # 1 because we are adding the header of raw in the 1st row
            valueInputOption="RAW",
            body={"values": [[raw_header]]}
        ).execute()
        time.sleep(RATE_LIMIT_DELAY)

        # Add formulas to the new column
        for i in range(2, 11):
            if i == 9:
                formula = f'={col_to_update}7-{col_to_update}8'
            elif i == 10:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)/60'
            else:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i+1}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)/60'

            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{col_to_update}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()
            time.sleep(RATE_LIMIT_DELAY)

def main():
    service = get_sheets_service()
    if service:
        create_or_update_sheet(service, SPREADSHEET_ID, TARGET_SHEET_NAME)

if __name__ == "__main__":
    main()


# adding hours column to batchaudiosummary

In [1]:
import os
import time
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# Parameters
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = "1Esy8YW-t2B3-8uwhaFrDD2-1fgzcXJ31hyi97DVPKXI"  # raw sheet to update from
CREDENTIALS_FILE = "client_secret_70065301055-fjcq0nugl0id8k0oab6qjse8e3o24in6.apps.googleusercontent.com.json"
TOKEN_FILE = "token.json"
RAW_SHEET_NAME = "rawAuto"
TARGET_SHEET_NAME = "BatchAudioSummaryAuto"
START_FORMULA_COLUMN = ord('E')  # Start column 'E' in ASCII
END_FORMULA_COLUMN = ord('Z')  # End column 'Z' in ASCII
START_UPDATE_COLUMN = ord('C')  # Start column 'C' in ASCII
RATE_LIMIT_DELAY = 0.5  # Delay in seconds between requests

STATUS_DATA = [
    'Raw Delivered',
    'Delivered greater than acceptance threshold',
    'Raw Redelivery',
    'Redelivered greater than acceptance threshold',
    'Accepted post Initial Check (file level)',
    'Accepted post Initial check (chunk level)',
    'Accepted post automated single audio check (chunk level)',
    'Delivered for manual QC',
    'Accepted post final single Audio Manual QC (chunk level)'
]

# Function to authenticate and create a Google Sheets API service instance
def get_sheets_service():
    creds = None
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        return service
    except HttpError as err:
        print(err)
        return None

# Function to check if a sheet exists
def sheet_exists(service, spreadsheet_id, sheet_name):
    try:
        sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        for sheet in sheets:
            if sheet.get("properties", {}).get("title") == sheet_name:
                return True
        return False
    except HttpError as err:
        print(err)
        return False

def create_or_update_sheet(service, spreadsheet_id, sheet_name):
    # Create a DataFrame with the status data
    df = pd.DataFrame({'Status': STATUS_DATA})

    # Convert DataFrame to list of lists
    values = [df.columns.tolist()] + df.values.tolist()

    # Check if the sheet exists and create if not
    if not sheet_exists(service, spreadsheet_id, sheet_name):
        add_sheet_request = {
            "requests": [
                {
                    "addSheet": {
                        "properties": {
                            "title": sheet_name
                        }
                    }
                }
            ]
        }
        service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=add_sheet_request).execute()

    # Write the DataFrame to the sheet
    body = {
        "values": values
    }
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!A1",
        valueInputOption="RAW",
        body=body
    ).execute()
    time.sleep(RATE_LIMIT_DELAY)

    # Determine the number of columns in the 'raw' sheet dynamically
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=f"{RAW_SHEET_NAME}!2:2"  # Read the second row to get column headers
    ).execute()
    raw_headers = result.get("values", [])
    if raw_headers:
        num_columns_raw = len(raw_headers[0])
    else:
        num_columns_raw = 0

    # Loop through columns from 'E' to 'Z' for FORMULA_COLUMN and 'C' onwards for COL_TO_UPDATE
    for col_offset in range(min(END_FORMULA_COLUMN - START_FORMULA_COLUMN + 1, num_columns_raw)):
        formula_column = chr(START_FORMULA_COLUMN + col_offset)
        col_to_update = chr(START_UPDATE_COLUMN + col_offset)

        # Read the header of the formula column from the 'raw' sheet
        result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=f"{RAW_SHEET_NAME}!{formula_column}2"  # 2 because in raw sheet the header is in the 2nd row
        ).execute()
        raw_header = result.get("values", [[]])[0][0] if result.get("values") and len(result.get("values")) > 0 else ""

        # Update the header for the new column in 'BatchAudioSummary'
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!{col_to_update}1",  # 1 because we are adding the header of raw in the 1st row
            valueInputOption="RAW",
            body={"values": [[raw_header]]}
        ).execute()
        time.sleep(RATE_LIMIT_DELAY)

        # Add formulas to the new column
        for i in range(2, 11):
            if i == 9:
                formula = f'={col_to_update}7-{col_to_update}8'
            elif i == 10:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i}:{formula_column}1000)/60'
            else:
                formula = f'=SUMPRODUCT((MOD(ROW({RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)-ROW({RAW_SHEET_NAME}!{formula_column}{i+1}),8)=0)*{RAW_SHEET_NAME}!{formula_column}{i+1}:{formula_column}1000)/60'

            service.spreadsheets().values().update(
                spreadsheetId=spreadsheet_id,
                range=f"{sheet_name}!{col_to_update}{i}",
                valueInputOption="USER_ENTERED",
                body={"values": [[formula]]}
            ).execute()
            time.sleep(RATE_LIMIT_DELAY)

    # Add a new column for "# of Hours" with row sums
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f"{sheet_name}!B1",
        valueInputOption="RAW",
        body={"values": [["# of Hours"]]}

    ).execute()
    for i in range(2, 11):
        formula = f'=SUM({sheet_name}!C{i}:{chr(START_UPDATE_COLUMN + col_offset)}{i})'
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f"{sheet_name}!B{i}",
            valueInputOption="USER_ENTERED",
            body={"values": [[formula]]}
        ).execute()
        time.sleep(RATE_LIMIT_DELAY)


def main():
    service = get_sheets_service()
    if service:
        create_or_update_sheet(service, SPREADSHEET_ID, TARGET_SHEET_NAME)

if __name__ == "__main__":
    main()
